## Parse data files downloaded from Realtor.com

As part of this code we will parse the data files we have downloaded from the Realtor.com for the properties currently listed in Bay area.

Here we will scan all the files and load the data in normalized manner to the individual dataframes. 

Once we have the data model we can use this to dissect more into the data and learn more information about the current market.

### Import the libraries

In [1]:
import pandas as pd
import json
import os
from pandas import json_normalize

pd.set_option('display.max_rows', None)  # None means unlimited
pd.set_option('display.max_columns', None)  # None means unlimited
pd.set_option('display.max_colwidth', None)

## Parse the extracted data file from html page of property

### Function to return the json object or none for a .json file passes

In [2]:
def json_data_fetch(json_data):
    
    if json_data.get("props") is not None:
        if json_data.get("props").get("pageProps")is not None:
            if json_data.get("props").get("pageProps").get("initialReduxState") is not None:
                child_json = json_data.get("props").get("pageProps").get("initialReduxState").get("propertyDetails")

                prop_id_check = json_data.get("props").get("pageProps").get("initialReduxState").get("query")
                if prop_id_check is None:
                    return None
                else:
                    if prop_id_check.get('slug') is None:
                        return None
                    else:
                        property_id = prop_id_check.get('slug')[0]


                if child_json is not None:
                    home_description = child_json.get("description")
                    home_details = child_json.get("details")
                    flags = child_json.get("flags")
                    last_sold_date = child_json.get("last_sold_date")
                    listing_date = child_json.get("list_date")
                    list_price = child_json.get("list_price")
                    last_price_change_amount = child_json.get("last_price_change_amount")
                    local=child_json.get("local")
                    location = child_json.get("location")
                    if location is not None:
                        address = location.get("address")
                        neighborhoods = location.get("neighborhoods")
                        if address is not None:
                            city = address.get("city")
                        else:
                            city = None
                    else:
                        city = None
                        neighborhoods = None
                    #lat_long=child.json_data()
                    schools=child_json.get("nearby_schools")
                    sqft_price=child_json.get("price_per_sqft")
                    #hoa
                    property_hist=child_json.get("property_history")
                    status=child_json.get("status")
                    tags=child_json.get("tags")
                    tax_history=child_json.get("tax_history")

                    if child_json.get("location") is None:
                        lat_long = None
                    elif child_json.get("location").get("address") is None:
                        lat_long = None
                    else:
                        lat_long = child_json.get("location").get("address").get("coordinate")           

                    if child_json.get("hoa") is None:
                        hoa = None
                    else:
                        hoa = child_json.get("hoa").get("fee")

                    return {'property_id': property_id,
                            'home_description': home_description,
                            'home_details': home_details,
                            'flags': flags,
                            'last_sold_date': last_sold_date,
                            'listing_date': listing_date,
                            'list_price': list_price,
                            'last_price_change_amount': last_price_change_amount,
                            'local': local,
                            "location":location,
                            'lat_long': lat_long,
                            'schools': schools,
                            'sqft_price': sqft_price,
                            'hoa': hoa,
                            'property_hist': property_hist,
                            'status': status,
                            'tags': tags,
                            'tax_history': tax_history,
                            'city':city,
                            'neighborhoods':neighborhoods
                           }

                else:
                    return None
            else:
                return None
        else:
            return None

    else:
        return None

In [3]:
# Function to process the json files we have downloaded and return if we have any issues with ethe json file downloaded

def return_json_object(file_name):
    try:
        with open(file_name,'r',encoding='utf-8') as json_file:
            json_data = json.load(json_file)
            return json_data
    except Exception as e:
        print(f"Issue with json data processing {file_name} : {str(e)}")
        return None

In [4]:
## Normalize the home Details for processing the home details dataframe 

def norm_home_details_df(df):
    
    property_ids = []
    categories = []
    parent_category = []
    texts = []
    
    for index, row in df.iterrows():
        property_id = row['property_id']
        home_details = row['home_details']
        
        for detail in home_details:
            property_ids.append(property_id)
            categories.append(detail['category'])
            parent_category.append(detail['parent_category'])
            texts.append(','.join(detail['text']))
    
    normalized_df = pd.DataFrame({
        'property_id':property_ids,
        'category':categories,
        'parent_category':parent_category,
        'text':texts
    })
    
    return normalized_df

In [5]:
## Normalize the local data to capture the noise in columns instead of a list 

def norm_local(row):
    noise_score = row['noise']['score']
    
    airport_noise = next((item['text'] for item in row['noise']['noise_categories'] if item['type'] == 'airport'), None)
    traffic_noise = next((item['text'] for item in row['noise']['noise_categories'] if item['type'] == 'traffic'), None)
    local_noise = next((item['text'] for item in row['noise']['noise_categories'] if item['type'] == 'local'), None)
    overall_noise = next((item['text'] for item in row['noise']['noise_categories'] if item['type'] == 'score'), None)
    
    return pd.Series([noise_score, airport_noise, traffic_noise, local_noise, overall_noise],
            index=['noise_score', 'airport_noise', 'traffic_noise', 'local_noise', 'overall_noise'])

In [6]:
# NOrmalize the schoolds rating data 

def norm_schools(df):
    
    property_ids = []
    level = []
    school_type = []
    rating = []
    parent_rating = []
    distance = []
    
    for index, row in df.iterrows():
        property_id = row['property_id']
        schools_data = row.get('schools')
        if schools_data and 'schools' in schools_data:
            school_details = schools_data['schools']
            
            for schl in school_details:
                property_ids.append(property_id)
                school_type.append(schl.get('funding_type'))
                rating.append(schl.get('rating'))
                parent_rating.append(schl.get('parent_rating'))
                distance.append(schl.get('distance_in_miles'))
                level.append(schl.get('education_levels'))
    
    normalized_df = pd.DataFrame({
        'property_id':property_ids,
        'school_type':school_type,
        'rating':rating,
        'parent_rating':parent_rating,
        'distance':distance,
        'level':level
        
    })
    
    return normalized_df

In [7]:
## Get the zip Code of teh propoerty from the propeorty link

def zip_codes(row):
    return row.split("CA")[1].split("_")[1]


In [8]:
## Extract the median neighborhood price for the data field

def extract_median_neighborhood_price(row):
    if row is None:
        return None
    else:
        for item in row:
            if item.get('level') == 'neighborhood':
                if item.get('geo_statistics') is not None:
                    if item.get('geo_statistics').get('housing_market') is not None:
                        return item.get('geo_statistics').get('housing_market').get('median_listing_price')
                
        return None       
    

### Define and initialize the variables

Here let's define the variables needed for the code execution


In [9]:
property_data_dir = 'property_data/' # Directory where all the json files are available. 


In [10]:
## Iterate the files in the data directory and load the data in the dataframe
files = os.listdir(property_data_dir)  # List the files in the directory. 
# Iterate over the files and get the json object in return whcih we will pass it to the function to fetch the 
# property details in the form of key_dict, to be used in the dataframe and data model creation later
dfs = []
for file in files:
    if file.endswith('.json'):
        file_name = os.path.join(property_data_dir,file)
        json_data = return_json_object(file_name)
        
        if json_data is None:
            print("Issue with the data processing of the ", file_name)
        else:
            property_dict = json_data_fetch(json_data)
            if property_dict is not None:
                df = pd.DataFrame([property_dict]) 
                dfs.append(df)

data = pd.concat(dfs, ignore_index=True)
data["zipcode"]= data["property_id"].apply(zip_codes)
data["median_nieghborhood_price"] = data['neighborhoods'].apply(extract_median_neighborhood_price)
data.drop(columns = ['neighborhoods'],inplace= True)
#data['neighborhoods'].isna().any()
data.head(1)

property_id  \
0  601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        home_description  \
0  {'baths': 2, 'baths_3qtr': None, 'baths_full': 2, 'baths_half': None, 'baths_max': None, 'baths_min': None, 'baths_total': None, 'baths_consolidated': '2', 'beds': 2, 'beds_max': None, 'beds_min': None, 'construction': None, 'cooling': None, 'exterior': None, 'fireplace': None, 'garage': None, 'garage_max': None, 'garage_min': None, 'garage_type': None, 'heating': None, 'logo': None, 'lot_sqft': 105807, 'name': None, 'pool': None, 'roofing': None, 'rooms': None, 'sqft': 1077, 'sqft_max': None, 'sqft_min': None, 'stories': 1, 'styles': None, 'sub_type': 'condo', 'text': 'Special extra large courtyard condo at Opera Plaza. Spacious two bedrooms and two bathrooms. Closets Galore! --all with custom closet buildouts. Super tranquil location on the courtyard gets sunshine all day! Flooded with natural light from the east and south. Lovely wood flooring throughout. Quality woodwork and built-ins including a massive wall of shelving and crown molding. HOA has installed new double pane windows (with screens) throughout. Two full bathrooms. Kitchen with stainless appliances. High service HOA with 24 hour security personnel, on-site management, 24 hour heated pool and spa, fitness, racquet courts, concierge and more! HOA fee includes amenities plus water, trash, high-speed fiber internet, building maintenance and building EQ insurance. On-site leased parking (optional).', 'type': 'condos', 'units': None, 'year_built': 1982, 'year_renovated': None, 'zoning': None, '__typename': 'HomeDescription'}   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [11]:
home_desc_df = pd.concat([data[['property_id']],json_normalize(data['home_description'])],axis =1).drop(['__typename'],axis=1)
home_desc_df.head()

,property_id,baths,baths_3qtr,baths_full,baths_half,baths_max,baths_min,baths_total,baths_consolidated,beds,beds_max,beds_min,construction,cooling,exterior,fireplace,garage,garage_max,garage_min,garage_type,heating,logo,lot_sqft,name,pool,roofing,rooms,sqft,sqft_max,sqft_min,stories,styles,sub_type,text,type,units,year_built,year_renovated,zoning
0,601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846,2.0,None,2.0,NaN,None,None,NaN,2,2.0,None,None,None,None,None,None,NaN,None,None,None,None,None,105807.0,None,None,None,NaN,1077.0,None,None,1.0,None,condo,"Special extra large courtyard condo at Opera Plaza. Spacious two bedrooms and two bathrooms. Closets Galore! --all with custom closet buildouts. Super tranquil location on the courtyard gets sunshine all day! Flooded with natural light from the east and south. Lovely wood flooring throughout. Quality woodwork and built-ins including a massive wall of shelving and crown molding. HOA has installed new double pane windows (with screens) throughout. Two full bathrooms. Kitchen with stainless appliances. High service HOA with 24 hour security personnel, on-site management, 24 hour heated pool and spa, fitness, racquet courts, concierge and more! HOA fee includes amenities plus water, trash, high-speed fiber internet, building maintenance and building EQ insurance. On-site leased parking (optional).",condos,NaN,1982.0,NaN,None
1,1762-Scott-St_San-Jose_CA_95128_M14724-99997,2.0,None,2.0,NaN,None,None,NaN,2,3.0,None,None,None,None,None,None,2.0,None,None,None,None,None,6098.0,None,None,None,NaN,1161.0,None,None,1.0,None,None,"Nestled in a desirable location close to Valley Fair, Santana Row, and the vibrant Downtown San Jose, this charming single-family house is a gem waiting to be discovered. Functional and airy floor plan that seamlessly integrates the living, dining, and kitchen areas. The updated kitchen boasts modern appliances and ample storage, while the renovated bathrooms offer a touch of luxury and convenience.Natural light floods the interior, creating a warm and inviting ambiance throughout the home. With three bedrooms and two bathrooms, there's plenty of space for the whole family to spread out and relax. Plus, the possibility of an ADU on the large lot opens up a world of potential for additional living space or rental income. Stay cool and comfortable with the central A/C system, while the owned solar panels help reduce energy costs. Convenience is key with this property, offering easy access to major highways such as 280, 880, and 17, as well as proximity to shopping centers, restaurants, and employment opportunities. Whether you're commuting to work or exploring the vibrant city life, everything you need is just moments away.",single_family,NaN,1950.0,NaN,None
2,32529-Gina-Way_Union-City_CA_94587_M15996-51300,2.0,None,2.0,NaN,None,None,NaN,2,3.0,None,None,None,None,None,None,2.0,None,None,None,None,None,6000.0,None,None,None,NaN,1320.0,None,None,1.0,None,None,"O/H Sat/Sun 1pm-4pm. Located in a quiet and desirable Union City neighborhood, this 3-bedroom, 2-bathroom single-level home offers 1, 320 square feet of living space on a spacious 6, 000 square foot lot. With its well-designed layout and ample outdoor area, it's perfect for comfortable single-family living and outdoor activities. The updated wood laminate flooring throughout most of the home exudes a clean and modern ambiance, while recessed lighting warmly illuminates the living spaces. The spacious living room features a large window allowing plenty of natural light to flood and brighten the space. The kitchen boasts Quartz countertops and backsplash, electric stove cooktop, updated stainless steel appliances including a built-in oven and microwave. Family room which includes a wood burning fireplace adorned with brick furnishings and details alongside a large sliding glass door that floods the space with ample amounts of sunlight. The primary bathroom features an exquisitely renovated shower with a g

In [12]:
home_details_df = norm_home_details_df(data[['property_id','home_details']])

In [13]:
# Column Names for your built-

# Type- Condos
# Price. 
# City Name - are there 
# Bathrooms
# Bedrooms
# sqft size
# Lot sqft Size
# listing date
# zipcode
# Average school rating 
# Built Year

In [14]:
flags_df = pd.concat([data[['property_id']],json_normalize(data['flags'])],axis =1).drop(['__typename'],axis=1)
flags_df.head()

,property_id,is_coming_soon,is_contingent,is_deal_available,is_for_rent,is_foreclosure,is_garage_present,is_new_construction,is_pending,is_price_excludes_land,is_senior_community,is_short_sale,is_subdivision,is_price_reduced,is_new_listing
0,601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846,None,None,None,None,None,None,None,None,None,None,None,None,None,False
1,1762-Scott-St_San-Jose_CA_95128_M14724-99997,None,None,None,None,None,True,None,None,None,None,None,None,None,False
2,32529-Gina-Way_Union-City_CA_94587_M15996-51300,None,None,None,None,None,True,None,True,None,None,None,None,None,False
3,9770-Via-Cantera-Rd_Kenwood_CA_95452_M22170-85999,None,None,None,None,None,True,None,True,None,None,None,None,None,False
4,263-Llano-De-Los-Robles-Ave-Unit-4_San-Jose_CA_95136_M99720-83135,None,True,None,None,None,True,None,None,None,None,None,None,None,False


In [15]:
## Build the Local Dataframe for noise information

local_df = data[['property_id','local']]
local_df[['noise_score', 'airport_noise', 'traffic_noise', 'local_noise', 'overall_noise']] = local_df['local'].apply(norm_local).copy()
local_df.drop(columns=['local'], inplace=True)
local_df.head()

/var/folders/t6/62p5k1w921vdm0907xrsq0ph0000gn/T/ipykernel_13657/3052318395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  local_df[['noise_score', 'airport_noise', 'traffic_noise', 'local_noise', 'overall_noise']] = local_df['local'].apply(norm_local).copy()
/var/folders/t6/62p5k1w921vdm0907xrsq0ph0000gn/T/ipykernel_13657/3052318395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  local_df[['noise_score', 'airport_noise', 'traffic_noise', 'local_noise', 'overall_noise']] = local_df['local'].apply(n

,property_id,noise_score,airport_noise,traffic_noise,local_noise,overall_noise
0,601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846,56.0,Low,High,High,High
1,1762-Scott-St_San-Jose_CA_95128_M14724-99997,67.0,Medium,High,Medium,High
2,32529-Gina-Way_Union-City_CA_94587_M15996-51300,78.0,Low,Medium,Low,Medium
3,9770-Via-Cantera-Rd_Kenwood_CA_95452_M22170-85999,100.0,N/A,N/A,N/A,N/A
4,263-Llano-De-Los-Robles-Ave-Unit-4_San-Jose_CA_95136_M99720-83135,80.0,Low,Medium,Low,Medium


### Let's build the school dataframe to understnad the school's rating and information

In [16]:
schools_df = norm_schools(data[['property_id','schools']])
schools_df.head()

,property_id,school_type,rating,parent_rating,distance,level
0,601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846,public,4.0,5.0,0.5,[elementary]
1,601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846,public,7.0,3.0,3.7,[middle]
2,601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846,public,3.0,4.0,0.1,[elementary]
3,601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846,public,2.0,NaN,0.1,"[middle, high]"
4,601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846,private,NaN,4.0,0.2,[high]


In [17]:
mean_ratings = schools_df.groupby(['property_id','school_type'])['rating'].transform('mean')
mean_parent_ratings = schools_df.groupby(['property_id','school_type'])['parent_rating'].transform('mean')
mean_ratings_1 = schools_df.groupby(['property_id'])['rating'].transform('mean')
mean_parent_ratings_1 = schools_df.groupby(['property_id'])['parent_rating'].transform('mean')

schools_df['parent_rating'] = schools_df['parent_rating'].fillna(mean_parent_ratings)
schools_df['rating'] = schools_df['rating'].fillna(mean_ratings)

schools_df['parent_rating'] = schools_df['parent_rating'].fillna(schools_df['rating'])
schools_df['rating'] = schools_df['rating'].fillna(schools_df['parent_rating'])


schools_df['parent_rating'] = schools_df['parent_rating'].fillna(mean_parent_ratings_1)
schools_df['rating'] = schools_df['rating'].fillna(mean_ratings_1)
schools_df.head()

,property_id,school_type,rating,parent_rating,distance,level
0,601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846,public,4.0,5.0,0.5,[elementary]
1,601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846,public,7.0,3.0,3.7,[middle]
2,601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846,public,3.0,4.0,0.1,[elementary]
3,601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846,public,2.0,4.0,0.1,"[middle, high]"
4,601-Van-Ness-Ave-Apt-125_San-Francisco_CA_94102_M20674-38846,private,4.0,4.0,0.2,[high]


In [18]:
public_elem_parent_rating = schools_df[(schools_df['level'].apply(lambda x: 'elementary' in x)) & (schools_df['school_type'] =='public')].groupby('property_id')['parent_rating'].mean()
public_elem_rating = schools_df[(schools_df['level'].apply(lambda x: 'elementary' in x)) & (schools_df['school_type'] =='public')].groupby('property_id')['rating'].mean()
private_elem_parent_rating = schools_df[(schools_df['level'].apply(lambda x: 'elementary' in x)) & (schools_df['school_type'] =='private')].groupby('property_id')['parent_rating'].mean()
private_elem_rating = schools_df[(schools_df['level'].apply(lambda x: 'elementary' in x)) & (schools_df['school_type'] =='private')].groupby('property_id')['rating'].mean()

# middle School
public_mid_parent_rating = schools_df[(schools_df['level'].apply(lambda x: 'middle' in x)) & (schools_df['school_type'] =='public')].groupby('property_id')['parent_rating'].mean()
public_mid_rating = schools_df[(schools_df['level'].apply(lambda x: 'middle' in x)) & (schools_df['school_type'] =='public')].groupby('property_id')['rating'].mean()
private_mid_parent_rating = schools_df[(schools_df['level'].apply(lambda x: 'middle' in x)) & (schools_df['school_type'] =='private')].groupby('property_id')['parent_rating'].mean()
private_mid_rating = schools_df[(schools_df['level'].apply(lambda x: 'middle' in x)) & (schools_df['school_type'] =='private')].groupby('property_id')['rating'].mean()

# high School
public_high_parent_rating = schools_df[(schools_df['level'].apply(lambda x: 'high' in x)) & (schools_df['school_type'] =='public')].groupby('property_id')['parent_rating'].mean()
public_high_rating = schools_df[(schools_df['level'].apply(lambda x: 'high' in x)) & (schools_df['school_type'] =='public')].groupby('property_id')['rating'].mean()
private_high_parent_rating = schools_df[(schools_df['level'].apply(lambda x: 'high' in x)) & (schools_df['school_type'] =='private')].groupby('property_id')['parent_rating'].mean()
private_high_rating = schools_df[(schools_df['level'].apply(lambda x: 'high' in x)) & (schools_df['school_type'] =='private')].groupby('property_id')['rating'].mean()
nearest_school = schools_df.groupby('property_id')['distance'].min()

final_schools_df = pd.DataFrame({
    'public_elem_parent_rating':public_elem_parent_rating,
    'public_elem_rating':public_elem_rating,
    'private_elem_parent_rating':private_elem_parent_rating,
    'private_elem_rating':private_elem_rating,
    'public_mid_parent_rating':public_mid_parent_rating,
    'public_mid_rating':public_mid_rating,
    'private_mid_parent_rating':private_mid_parent_rating,
    'private_mid_rating':private_mid_rating,
    'public_high_parent_rating':public_high_parent_rating,
    'public_high_rating':public_high_rating,
    'private_high_parent_rating':private_high_parent_rating,
    'private_high_rating':private_high_rating,
    'nearest_school':nearest_school })

final_schools_df.reset_index(inplace=True)
final_schools_df.head()


,property_id,public_elem_parent_rating,public_elem_rating,private_elem_parent_rating,private_elem_rating,public_mid_parent_rating,public_mid_rating,private_mid_parent_rating,private_mid_rating,public_high_parent_rating,public_high_rating,private_high_parent_rating,private_high_rating,nearest_school
0,1-Baldwin-Ave-Apt-516_San-Mateo_CA_94401_M29122-59505,4.208333,5.833333,4.333333,4.333333,3.944444,6.444444,4.0,4.0,3.611111,7.444444,NaN,NaN,0.1
1,1-Baldwin-Ave-Apt-905_San-Mateo_CA_94401_M14325-89364,4.208333,5.833333,4.500000,4.500000,3.944444,6.444444,4.5,4.5,3.611111,7.444444,5.0,5.0,0.1
2,1-Bristol-Ct-Apt-201_San-Francisco_CA_94130_M94079-93440,4.244444,6.250000,NaN,NaN,4.055556,5.156250,3.0,3.0,4.305556,2.656250,3.0,3.0,1.0
3,1-Daniel-Burnham-Ct-Apt-324_San-Francisco_CA_94109_M23769-89053,4.333333,5.000000,4.500000,4.500000,4.133333,4.333333,5.0,5.0,4.133333,4.666667,4.0,4.0,0.1
4,1-Hawthorne-St-Unit-10D_San-Francisco_CA_94105_M15514-07280,4.666667,3.190476,4.500000,4.500000,4.222222,2.333333,4.5,4.5,4.833333,2.000000,4.5,4.5,0.3


In [19]:
#Let's build the final data frame to ingest into the excel file for analysis
selected_cols_list = ['property_id','last_sold_date','listing_date','list_price', 'last_price_change_amount', 'sqft_price',
                     'hoa','status','city','zipcode','median_nieghborhood_price','baths_consolidated','beds','garage',
                     'lot_sqft','pool','sqft','stories','type','public_elem_parent_rating','public_elem_rating',
                     'private_elem_parent_rating','private_elem_rating','public_mid_parent_rating',
                     'public_mid_rating','private_mid_parent_rating','private_mid_rating','public_high_parent_rating',
                     'public_high_rating','private_high_parent_rating','private_high_rating','nearest_school',
                     'noise_score','airport_noise','traffic_noise','local_noise','overall_noise',
                     'is_foreclosure','is_new_construction','is_pending','is_senior_community','is_new_listing'
                     ]
final_data_df = pd.merge(data,home_desc_df, on = 'property_id', how = 'inner')
final_data_df1 = pd.merge(final_data_df,final_schools_df, on = 'property_id', how = 'inner')
final_data_df2 = pd.merge(final_data_df1,local_df, on ='property_id', how = 'inner')
final_data_df3 = pd.merge(final_data_df2,flags_df, on = 'property_id',how = 'inner')
final_df = final_data_df3[selected_cols_list]


# Some data cleaning and re-assignment
final_df['is_senior_community'] = final_df['is_senior_community'].fillna(False)
final_df['is_pending'] = final_df['is_pending'].fillna(False)
final_df['is_new_construction'] = final_df['is_new_construction'].fillna(False)
final_df['is_foreclosure'] = final_df['is_foreclosure'].fillna(False)
final_df['stories'] = final_df['stories'].fillna(0)
final_df['hoa'] = final_df['hoa'].fillna(0)
final_df['garage'] = final_df['garage'].fillna(0)
final_df['type'] = final_df['type'].replace('condos','condo')
final_df['type'] = final_df['type'].replace('townhomes','townhouse')
final_df['type'] = final_df['type'].replace('condo_townhome','townhouse')
final_df['type'] = final_df['type'].replace('condo_townhome_rowhome_coop','townhouse')
final_df.type.unique()


/var/folders/t6/62p5k1w921vdm0907xrsq0ph0000gn/T/ipykernel_13657/1260517310.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['is_senior_community'] = final_df['is_senior_community'].fillna(False)
/var/folders/t6/62p5k1w921vdm0907xrsq0ph0000gn/T/ipykernel_13657/1260517310.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['is_pending'] = final_df['is_pending'].fillna(False)
/var/folders/t6/62p5k1w921vdm0907xrsq0ph0000gn/T/ipykernel_13657/1260517310.py:19: SettingWithCopyWarning: 
A va

array(['condo', 'single_family', 'townhouse', 'mobile', 'land',
       'multi_family', 'farm', None], dtype=object)

In [20]:
# Correct the property details whose information was missing in the data during the EDA. 
final_df.loc[final_df['property_id'] == '1842-Church-St_San-Francisco_CA_94131_M28833-11571','beds'] = 2.0
final_df[final_df['property_id'] == '1842-Church-St_San-Francisco_CA_94131_M28833-11571']

,property_id,last_sold_date,listing_date,list_price,last_price_change_amount,sqft_price,hoa,status,city,zipcode,median_nieghborhood_price,baths_consolidated,beds,garage,lot_sqft,pool,sqft,stories,type,public_elem_parent_rating,public_elem_rating,private_elem_parent_rating,private_elem_rating,public_mid_parent_rating,public_mid_rating,private_mid_parent_rating,private_mid_rating,public_high_parent_rating,public_high_rating,private_high_parent_rating,private_high_rating,nearest_school,noise_score,airport_noise,traffic_noise,local_noise,overall_noise,is_foreclosure,is_new_construction,is_pending,is_senior_community,is_new_listing
810,1842-Church-St_San-Francisco_CA_94131_M28833-11571,1979-06-14,2024-04-20T22:58:29Z,2980850,None,1129,0,for_sale,San Francisco,94131,1850000.0,4,2.0,2.0,3123.0,None,2640.0,3.0,single_family,3.9,3.2,4.333333,4.333333,3.25,2.75,4.5,4.5,2.666667,6.0,NaN,NaN,0.1,72.0,Low,Medium,Medium,Medium,False,False,False,False,True


In [21]:
## Load the data to the data files, for use in the EDA

final_df.to_excel("merged_data.xlsx", index=False)
final_df.to_csv("merged_data.csv", index=False)

I have imported the data for the median household income to understand the current real estate family homes and understand if the current real estate market supports to what %age of the households.